In [1]:
%pip install openai

# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them! 

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent! 

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf). 

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [2]:
# uncomment this if running locally
#!pip install python-dotenv
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [1]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a joke about a bold teacher explaining agents to wonderful students.")
Markdown(output)

Why did the bold teacher bring a ladder to class while explaining agents? 

Because they wanted to ensure their students could reach new heights in understanding!

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [2]:
def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task? 

In the famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) they demonstrated that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

In [3]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content


Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function


This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [4]:
def get_response(prompt_question, model="gpt-3.5-turbo-16k"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(dir_name):
    os.makedirs(dir_name, exist_ok=True)

def create_file(filename):
    with open(filename, 'w'):
        pass

def list_files():
    files = os.listdir()
    for file in files:
        print(file)

    

task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder, create a file called 'the-10-master-rules.md"
output = get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:
                            
    def create_directory(dir_name):
        os.makedirs(dir_name, exist_ok=True)

    def create_file():
        with open('test.txt', 'w'):
            pass

    def list_files():
        files = os.listdir()
        for file in files:
            print(file)
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

create_directory('lucas-the-agent-master')

At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [ ]:
import json
import subprocess
import json
import os

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    os.mkdir(directory_name)
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]

In [ ]:
import json

def run_terminal_task():
    messages = [
        {"role": "user", 
         "content": "Create a folder called 'pancakes-are-better-than-waffles'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-9mIEyVjdolDXumxL2BRsIlKI3ttWM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I have created a folder called 'pancakes-are-better-than-waffles'.", role='assistant', function_call=None, tool_calls=None))], created=1721296888, model='gpt-3.5-turbo-16k-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=69, total_tokens=86))

In [ ]:
output.choices[0].message.content

"I have created a folder called 'pancakes-are-better-than-waffles'."

In [ ]:
!ls -d */

agent-deploy/                     lucas-the-agent-master/
assets-resources/                 pancakes-are-better-than-waffles/
lucas-agent-007/                  pancakes-rule/
lucas-loves-pancakes/             test/


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)